In [8]:
!pip install sqlalchemy pymysql -q

In [1]:
!pip install sqlalchemy sql_magic -q

In [2]:
%load_ext sql_magic

/opt/conda/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
%load_ext sql_magic

The sql_magic extension is already loaded. To reload it, use:
  %reload_ext sql_magic


In [4]:
%sql

UsageError: Line magic function `%sql` not found.


In [5]:
import pandas as pd
import numpy as np

In [6]:
user="cornell"
password="Pitney@1a"
host="gec-dwh.ctz5iw1v2fh7.us-west-2.rds.amazonaws.com"
db="cornell"

conn = f"mysql+pymysql://{user}:{password}@{host}/{db}"

In [7]:
from sqlalchemy import create_engine

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db}")

In [8]:
result = engine.execute('show databases')
for _r in result:
    print(_r)

('information_schema',)
('cornell',)


In [22]:
result = engine.execute('show tables in cornell')
for _r in result:
    print(_r)

('deliver_package_info',)
('input_df',)
('ne_delivery_data_by_day_2018',)
('ne_returns_data_by_day_2018',)
('return_package_info',)
('return_package_info_update',)
('test_df',)


## Return null zips

In [23]:
result = engine.execute('select originfacilityzip,destinationfacilityzip from cornell.ne_returns_data_by_day_2018 where originfacilityzip = "''" or destinationfacilityzip="''" group by originfacilityzip,destinationfacilityzip')
for _r in result:
    print(_r)

('', '01460')
('', '02379')
('', '03053')
('', '07032')
('', '07094')
('', '07836')
('', '08817')
('', '08831')
('', '08837')
('', '11096')
('', '11102')
('', '11413')
('', '11514')
('', '14214')
('', '15672')
('', '16368')
('', '17022')
('', '17055')
('', '17402')
('', '17527')
('', '17554')
('', '17699')
('', '18015')
('', '18101')
('', '18640')
('', '18702')
('', '21057')
('', '21130')
('', '21704')
('', '21801')
('', '23230')
('', '27801')
('', '30349')
('', '46037')
('', '52404')
('', '75039')
('', '76177')
('', '92407')
('', '92551')
('', '93012')


## Delivery Null zips

In [24]:
result = engine.execute('select originfacilityzip,destinationfacilityzip from cornell.ne_delivery_data_by_day_2018 where originfacilityzip = "''" or destinationfacilityzip = "''" group by originfacilityzip,destinationfacilityzip')
for _r in result:
    print(_r)

('06037', '')
('06492', '')
('08837', '')
('11096', '')
('11111', '')
('15122', '')
('17015', '')
('17055', '')
('19605', '')
('30349', '')
('43068', '')
('46037', '')
('78746', '')
('\\N', '')


## Delivery Null Calculation

In [9]:
result = engine.execute('select count(*) from cornell.ne_delivery_data_by_day_2018')
totalDelPackage_df = pd.DataFrame(result.fetchall(),columns=['totalcount'])
totalDelPackage_df

,totalcount
0,61916290


In [10]:
result = engine.execute('select count(*) from cornell.ne_delivery_data_by_day_2018 where originfacilityzip = "''"')
oriNullDelPackage_df = pd.DataFrame(result.fetchall(),columns=['totalnullcount'])
oriNullDelPackage_df

,totalnullcount
0,0


In [11]:
result = engine.execute('select count(*) from cornell.ne_delivery_data_by_day_2018 where destinationfacilityzip = "''" ')
destNullDelPackage_df = pd.DataFrame(result.fetchall(),columns=['totalnullcount'])
destNullDelPackage_df

,totalnullcount
0,1195932


In [12]:
print(oriNullDelPackage_df['totalnullcount'][0]/totalDelPackage_df['totalcount'][0]*100)
print(destNullDelPackage_df['totalnullcount'][0]/totalDelPackage_df['totalcount'][0]*100)

0.0
1.9315304583


## Return null calculation

In [14]:
result = engine.execute('select count(*) from cornell.ne_returns_data_by_day_2018 ')
totalReturnPackage_df = pd.DataFrame(result.fetchall(),columns=['totalcount'])
totalReturnPackage_df 

,totalcount
0,33276614


In [16]:
result = engine.execute('select count(*) from cornell.ne_returns_data_by_day_2018 where originfacilityzip = "''" ')
oriNullReturnPackage_df = pd.DataFrame(result.fetchall(),columns=['nullcount'])
oriNullReturnPackage_df 


,nullcount
0,49696


In [17]:
result = engine.execute('select sum(count) from cornell.ne_returns_data_by_day_2018 where destfacilityzip = "''" ')
destNullReturnPackage_df = pd.DataFrame(result.fetchall(),columns=['nullcount'])
destNullReturnPackage_df 

InternalError: (pymysql.err.InternalError) (1054, "Unknown column 'count' in 'field list'") [SQL: 'select sum(count) from cornell.ne_returns_data_by_day_2018 where destfacilityzip = "" ']

In [ ]:
print(oriNullReturnPackage_df['nullcount'][0]/totalReturnPackage_df['totalcount'][0]*100)